In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
pd.options.mode.chained_assignment = None

In [2]:
df = pd.read_csv(r'dataset/cloudwatch/CloudWatch_Traffic_Web_Attack.csv')

In [3]:
df.head()

,bytes_in,bytes_out,creation_time,end_time,src_ip,src_ip_country_code,protocol,response.code,dst_port,dst_ip,rule_names,observation_name,source.meta,source.name,time,detection_types
0,5602,12990,2024-04-25T23:00:00Z,2024-04-25T23:10:00Z,147.161.161.82,AE,HTTPS,200,443,10.138.69.97,Suspicious Web Traffic,Adversary Infrastructure Interaction,AWS_VPC_Flow,prod_webserver,2024-04-25T23:00:00Z,waf_rule
1,30912,18186,2024-04-25T23:00:00Z,2024-04-25T23:10:00Z,165.225.33.6,US,HTTPS,200,443,10.138.69.97,Suspicious Web Traffic,Adversary Infrastructure Interaction,AWS_VPC_Flow,prod_webserver,2024-04-25T23:00:00Z,waf_rule
2,28506,13468,2024-04-25T23:00:00Z,2024-04-25T23:10:00Z,165.225.212.255,CA,HTTPS,200,443,10.138.69.97,Suspicious Web Traffic,Adversary Infrastructure Interaction,AWS_VPC_Flow,prod_webserver,2024-04-25T23:00:00Z,waf_rule
3,30546,14278,2024-04-25T23:00:00Z,2024-04-25T23:10:00Z,136.226.64.114,US,HTTPS,200,443,10.138.69.97,Suspicious Web Traffic,Adversary Infrastructure Interaction,AWS_VPC_Flow,prod_webserver,2024-04-25T23:00:00Z,waf_rule
4,6526,13892,2024-04-25T23:00:00Z,2024-04-25T23:10:00Z,165.225.240.79,NL,HTTPS,200,443,10.138.69.97,Suspicious Web Traffic,Adversary Infrastructure Interaction,AWS_VPC_Flow,prod_webserver,2024-04-25T23:00:00Z,waf_rule


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282 entries, 0 to 281
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   bytes_in             282 non-null    int64 
 1   bytes_out            282 non-null    int64 
 2   creation_time        282 non-null    object
 3   end_time             282 non-null    object
 4   src_ip               282 non-null    object
 5   src_ip_country_code  282 non-null    object
 6   protocol             282 non-null    object
 7   response.code        282 non-null    int64 
 8   dst_port             282 non-null    int64 
 9   dst_ip               282 non-null    object
 10  rule_names           282 non-null    object
 11  observation_name     282 non-null    object
 12  source.meta          282 non-null    object
 13  source.name          282 non-null    object
 14  time                 282 non-null    object
 15  detection_types      282 non-null    object
dtypes: int64

In [5]:
# time column seems to be duplicate or unnecessary, will drop it
df = df.drop(columns=['time'])

In [6]:
df.head(2)

,bytes_in,bytes_out,creation_time,end_time,src_ip,src_ip_country_code,protocol,response.code,dst_port,dst_ip,rule_names,observation_name,source.meta,source.name,detection_types
0,5602,12990,2024-04-25T23:00:00Z,2024-04-25T23:10:00Z,147.161.161.82,AE,HTTPS,200,443,10.138.69.97,Suspicious Web Traffic,Adversary Infrastructure Interaction,AWS_VPC_Flow,prod_webserver,waf_rule
1,30912,18186,2024-04-25T23:00:00Z,2024-04-25T23:10:00Z,165.225.33.6,US,HTTPS,200,443,10.138.69.97,Suspicious Web Traffic,Adversary Infrastructure Interaction,AWS_VPC_Flow,prod_webserver,waf_rule


***

### Initialise Functions For Cleaning/Checking

In [7]:
# Function to strip strings, check for inconsistent nulls > np.nan > report total

def clean_and_check_nulls(dataframe):
    # Strip leading/trailing spaces for string columns
    dataframe = dataframe.applymap(lambda x: x.strip() if isinstance(x, str) else x)

    # Replace empty strings and common null-like values with np.nan
    dataframe.replace(['', 'NULL', 'null', 'N/A'], np.nan, inplace=True)

    # Check for null values in each column
    null_count = dataframe.isnull().sum()

    # Print out the column names and their respective null counts
    for col, nulls in null_count.items():
        if nulls > 0:
            print(f"Column '{col}' has {nulls} null value(s).")

    return dataframe

In [8]:
# Function to convert specified columns in the DataFrame to datetime64 type.

def convert_to_datetime(dataframe, columns):    

    for col in columns:
        dataframe[col] = pd.to_datetime(dataframe[col], errors='coerce')
        print(f"Column '{col}' converted to datetime.")
    
    return dataframe

In [9]:
# Function to check specific columns for duplicates.

def check_and_drop_duplicates(df, columns):
    """
    Check for duplicate combinations based on specific columns, 
    count them, and drop duplicates if found.
    
    Parameters:
    df (pd.DataFrame): The dataframe to check for duplicates.
    columns (list): List of columns to check for duplicate combinations.
    
    Returns:
    pd.DataFrame: Dataframe with duplicates dropped, if found.
    """
    # Check for duplicated combinations
    duplicates = df[df.duplicated(subset=columns, keep=False)]
    
    if not duplicates.empty:
        # Count the number of duplicated combinations
        duplicate_count = duplicates.shape[0]
        print(f"{duplicate_count} duplicated combinations found in columns: {columns}")
        
        # Drop duplicates and keep the first occurrence
        df_deduped = df.drop_duplicates(subset=columns)
        print(f"{duplicate_count} duplicated combinations dropped.")
        return df_deduped
    else:
        print("No duplicated combinations were found.")
        return df

In [10]:
# Function to get all unique values from all columns within dataframe.

def get_unique_values(df, columns=None):
    unique_values_dict = {}

    # If no columns are specified, check all columns
    columns_to_check = columns if columns else df.columns
    
    # Loop through the specified columns
    for column in columns_to_check:
        if column in df.columns:
            unique_values_dict[column] = df[column].unique().tolist()
        else:
            print(f"Column '{column}' not found in the dataframe")
    
    # Format the output for better readability
    for column, unique_values in unique_values_dict.items():
        print(f"Column '{column}': {unique_values}")
    
    return unique_values_dict

***

### Cleaning

In [11]:
df = clean_and_check_nulls(df) # no output = no nulls

In [12]:
df.head(2)

,bytes_in,bytes_out,creation_time,end_time,src_ip,src_ip_country_code,protocol,response.code,dst_port,dst_ip,rule_names,observation_name,source.meta,source.name,detection_types
0,5602,12990,2024-04-25T23:00:00Z,2024-04-25T23:10:00Z,147.161.161.82,AE,HTTPS,200,443,10.138.69.97,Suspicious Web Traffic,Adversary Infrastructure Interaction,AWS_VPC_Flow,prod_webserver,waf_rule
1,30912,18186,2024-04-25T23:00:00Z,2024-04-25T23:10:00Z,165.225.33.6,US,HTTPS,200,443,10.138.69.97,Suspicious Web Traffic,Adversary Infrastructure Interaction,AWS_VPC_Flow,prod_webserver,waf_rule


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282 entries, 0 to 281
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   bytes_in             282 non-null    int64 
 1   bytes_out            282 non-null    int64 
 2   creation_time        282 non-null    object
 3   end_time             282 non-null    object
 4   src_ip               282 non-null    object
 5   src_ip_country_code  282 non-null    object
 6   protocol             282 non-null    object
 7   response.code        282 non-null    int64 
 8   dst_port             282 non-null    int64 
 9   dst_ip               282 non-null    object
 10  rule_names           282 non-null    object
 11  observation_name     282 non-null    object
 12  source.meta          282 non-null    object
 13  source.name          282 non-null    object
 14  detection_types      282 non-null    object
dtypes: int64(4), object(11)
memory usage: 33.2+ KB


**To-do:**
1. Check for unique values in each columns, a second thorough check ensuring no NA values.
2. Convert 'creation_time' & 'end_time' into datetime data type.
3. Use ipaddress library to check 'src_ip' & 'dst_ip' to ensure valid IP addresses.
4. Make sure categorical fields: 'protocol', 'response.code' & 'rule_names' are standardized.

**#1. Check unique values in specific columns to ensure no NA or outlier values for categorical columns**

In [14]:
unique_values = get_unique_values(df, ['src_ip_country_code','protocol','response.code','dst_port','rule_names','observation_name','source.meta','source.name','detection_types'])

Column 'src_ip_country_code': ['AE', 'US', 'CA', 'NL', 'DE', 'AT', 'IL']
Column 'protocol': ['HTTPS']
Column 'response.code': [200]
Column 'dst_port': [443]
Column 'rule_names': ['Suspicious Web Traffic']
Column 'observation_name': ['Adversary Infrastructure Interaction']
Column 'source.meta': ['AWS_VPC_Flow']
Column 'source.name': ['prod_webserver']
Column 'detection_types': ['waf_rule']


**No outliers within the categorical columns. Proceed to #2.**

**#2. Convert 'creation_time' & 'end_time' into datetime data type**

*Remarks: The timestamp 2024-04-25T23:00:00Z is in ISO 8601 format, where Z stands for UTC (Coordinated Universal Time).*

In [15]:
# Convert the 'creation_time' & 'end_time' columns to pandas datetime format
df['creation_time'] = pd.to_datetime(df['creation_time'], format='%Y-%m-%dT%H:%M:%SZ')
df['end_time'] = pd.to_datetime(df['end_time'], format='%Y-%m-%dT%H:%M:%SZ')

In [16]:
df.head(2)

,bytes_in,bytes_out,creation_time,end_time,src_ip,src_ip_country_code,protocol,response.code,dst_port,dst_ip,rule_names,observation_name,source.meta,source.name,detection_types
0,5602,12990,2024-04-25 23:00:00,2024-04-25 23:10:00,147.161.161.82,AE,HTTPS,200,443,10.138.69.97,Suspicious Web Traffic,Adversary Infrastructure Interaction,AWS_VPC_Flow,prod_webserver,waf_rule
1,30912,18186,2024-04-25 23:00:00,2024-04-25 23:10:00,165.225.33.6,US,HTTPS,200,443,10.138.69.97,Suspicious Web Traffic,Adversary Infrastructure Interaction,AWS_VPC_Flow,prod_webserver,waf_rule


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282 entries, 0 to 281
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   bytes_in             282 non-null    int64         
 1   bytes_out            282 non-null    int64         
 2   creation_time        282 non-null    datetime64[ns]
 3   end_time             282 non-null    datetime64[ns]
 4   src_ip               282 non-null    object        
 5   src_ip_country_code  282 non-null    object        
 6   protocol             282 non-null    object        
 7   response.code        282 non-null    int64         
 8   dst_port             282 non-null    int64         
 9   dst_ip               282 non-null    object        
 10  rule_names           282 non-null    object        
 11  observation_name     282 non-null    object        
 12  source.meta          282 non-null    object        
 13  source.name          282 non-null  

**Both 'creation_time' & 'end_time' converted to datetime format. Proceed to #3.**

**#3. Use ipaddress library to check 'src_ip' & 'dst_ip' to ensure valid IP addresses**

*Remarks: Although we can also import re to use regular expressions, for simplicity, we can test out ipaddress module.*

In [18]:
import ipaddress

In [19]:
# Function to check & validate the IP addresses in dataframe

def is_valid_ip(ip_list):
    invalid_ip_list = []
    
    for ip in ip_list:
        try:
            ipaddress.ip_address(ip)
        except ValueError:
            invalid_ip_list.append(ip)
    return invalid_ip_list

In [20]:
# Get list of invalid SRC IPs
invalid_src_ips = is_valid_ip(df['src_ip'])

# Print out list of invalid SRC IPs
print("Invalid Source IPs:", invalid_src_ips)

Invalid Source IPs: []


In [21]:
# Get list of invalid DST IPs
invalid_dst_ips = is_valid_ip(df['dst_ip'])

# Print out list of invalid DST IPs
print("Invalid Destination IPs:", invalid_dst_ips)

Invalid Destination IPs: []


**Done checking 'src_ip' & 'dst_ip'. Proceed to #4.**

**#4. Make sure categorical fields: 'protocol', 'response.code' & 'rule_names' are standardized.**

*Remarks: We can re-run get_unique_values to double-check for consistency & standardisation.*

In [22]:
double_check_consistency = get_unique_values(df, ['protocol','response.code','rule_names'])

Column 'protocol': ['HTTPS']
Column 'response.code': [200]
Column 'rule_names': ['Suspicious Web Traffic']


**We only have one type of protocol: HTTPS, one type of response code: 200 and the only rule name is 'Suspicious Web Traffic'.**

***

### Transformation

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282 entries, 0 to 281
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   bytes_in             282 non-null    int64         
 1   bytes_out            282 non-null    int64         
 2   creation_time        282 non-null    datetime64[ns]
 3   end_time             282 non-null    datetime64[ns]
 4   src_ip               282 non-null    object        
 5   src_ip_country_code  282 non-null    object        
 6   protocol             282 non-null    object        
 7   response.code        282 non-null    int64         
 8   dst_port             282 non-null    int64         
 9   dst_ip               282 non-null    object        
 10  rule_names           282 non-null    object        
 11  observation_name     282 non-null    object        
 12  source.meta          282 non-null    object        
 13  source.name          282 non-null  

In [24]:
df.head(3)

,bytes_in,bytes_out,creation_time,end_time,src_ip,src_ip_country_code,protocol,response.code,dst_port,dst_ip,rule_names,observation_name,source.meta,source.name,detection_types
0,5602,12990,2024-04-25 23:00:00,2024-04-25 23:10:00,147.161.161.82,AE,HTTPS,200,443,10.138.69.97,Suspicious Web Traffic,Adversary Infrastructure Interaction,AWS_VPC_Flow,prod_webserver,waf_rule
1,30912,18186,2024-04-25 23:00:00,2024-04-25 23:10:00,165.225.33.6,US,HTTPS,200,443,10.138.69.97,Suspicious Web Traffic,Adversary Infrastructure Interaction,AWS_VPC_Flow,prod_webserver,waf_rule
2,28506,13468,2024-04-25 23:00:00,2024-04-25 23:10:00,165.225.212.255,CA,HTTPS,200,443,10.138.69.97,Suspicious Web Traffic,Adversary Infrastructure Interaction,AWS_VPC_Flow,prod_webserver,waf_rule


**To-do:**
1. Duration of connection time, using 'creation_time' & 'end_time' differences.
2. Total traffic volume, using SUM of 'bytes_in' & 'bytes_out'.

**#1. Get total duration of from 'creation_time' to 'end_time' as 'connection_time'.**

In [25]:
df['connection_time'] = (df['end_time'] - df['creation_time']).dt.total_seconds()

In [26]:
df.head()

,bytes_in,bytes_out,creation_time,end_time,src_ip,src_ip_country_code,protocol,response.code,dst_port,dst_ip,rule_names,observation_name,source.meta,source.name,detection_types,connection_time
0,5602,12990,2024-04-25 23:00:00,2024-04-25 23:10:00,147.161.161.82,AE,HTTPS,200,443,10.138.69.97,Suspicious Web Traffic,Adversary Infrastructure Interaction,AWS_VPC_Flow,prod_webserver,waf_rule,600.0
1,30912,18186,2024-04-25 23:00:00,2024-04-25 23:10:00,165.225.33.6,US,HTTPS,200,443,10.138.69.97,Suspicious Web Traffic,Adversary Infrastructure Interaction,AWS_VPC_Flow,prod_webserver,waf_rule,600.0
2,28506,13468,2024-04-25 23:00:00,2024-04-25 23:10:00,165.225.212.255,CA,HTTPS,200,443,10.138.69.97,Suspicious Web Traffic,Adversary Infrastructure Interaction,AWS_VPC_Flow,prod_webserver,waf_rule,600.0
3,30546,14278,2024-04-25 23:00:00,2024-04-25 23:10:00,136.226.64.114,US,HTTPS,200,443,10.138.69.97,Suspicious Web Traffic,Adversary Infrastructure Interaction,AWS_VPC_Flow,prod_webserver,waf_rule,600.0
4,6526,13892,2024-04-25 23:00:00,2024-04-25 23:10:00,165.225.240.79,NL,HTTPS,200,443,10.138.69.97,Suspicious Web Traffic,Adversary Infrastructure Interaction,AWS_VPC_Flow,prod_webserver,waf_rule,600.0


**#2. Get total traffic, using SUM of 'bytes_in' & 'bytes_out as 'total_bytes'.**

In [27]:
df['total_bytes'] = df['bytes_in'] + df['bytes_out']

In [28]:
df.head()

,bytes_in,bytes_out,creation_time,end_time,src_ip,src_ip_country_code,protocol,response.code,dst_port,dst_ip,rule_names,observation_name,source.meta,source.name,detection_types,connection_time,total_bytes
0,5602,12990,2024-04-25 23:00:00,2024-04-25 23:10:00,147.161.161.82,AE,HTTPS,200,443,10.138.69.97,Suspicious Web Traffic,Adversary Infrastructure Interaction,AWS_VPC_Flow,prod_webserver,waf_rule,600.0,18592
1,30912,18186,2024-04-25 23:00:00,2024-04-25 23:10:00,165.225.33.6,US,HTTPS,200,443,10.138.69.97,Suspicious Web Traffic,Adversary Infrastructure Interaction,AWS_VPC_Flow,prod_webserver,waf_rule,600.0,49098
2,28506,13468,2024-04-25 23:00:00,2024-04-25 23:10:00,165.225.212.255,CA,HTTPS,200,443,10.138.69.97,Suspicious Web Traffic,Adversary Infrastructure Interaction,AWS_VPC_Flow,prod_webserver,waf_rule,600.0,41974
3,30546,14278,2024-04-25 23:00:00,2024-04-25 23:10:00,136.226.64.114,US,HTTPS,200,443,10.138.69.97,Suspicious Web Traffic,Adversary Infrastructure Interaction,AWS_VPC_Flow,prod_webserver,waf_rule,600.0,44824
4,6526,13892,2024-04-25 23:00:00,2024-04-25 23:10:00,165.225.240.79,NL,HTTPS,200,443,10.138.69.97,Suspicious Web Traffic,Adversary Infrastructure Interaction,AWS_VPC_Flow,prod_webserver,waf_rule,600.0,20418


***

### Split Data into Fact & Dimension Tables

***Rationale**: Given the nature of this current dataset, which contains a combination of network traffic metadata, source/destination details, and security-related attributes, the data would be more suited to a dimensional modeling approach (i.e., star schema with fact and dimension tables).*

***Personal Opinion**: This approach could be ideal for cybersecurity threat analysis (OLAP) because it simplifies querying and reporting, especially for analyzing trends, aggregations, and patterns of suspicious behavior in SQL.*

**Dataset Content**

Each entry in the dataset represents a stream of traffic to a web server, including the following columns:

* bytes_in: Bytes received by the server.
* bytes_out: Bytes sent from the server.
* creation_time: Timestamp of when the record was created.
* end_time: Timestamp of when the connection ended.
* src_ip: Source IP address.
* src_ip_country_code: Country code of the source IP.
* protocol: Protocol used in the connection.
* response.code: HTTP response code.
* dst_port: Destination port on the server.
* dst_ip: Destination IP address.
* rule_names: Name of the rule that identified the traffic as suspicious.
* observation_name: Observations associated with the traffic.
* source.meta: Metadata related to the source.
* source.name: Name of the traffic source.
* time: Timestamp of the detected event.
* detection_types: Type of detection applied.

**Additional columns:**

* total_bytes: Total traffic from bytes_in & bytes_out.
* connection_time: Total connection time from creation_time to end_time.

**Fact Table: Traffic_fact**

* Consists of : bytes_in, bytes_out, creation_time, end_time, connection_time, total_bytes
* This will store the primary metrics (quantitative data) from each record. 
* It will reference the respective dimension tables via foreign keys (FKEY).

In [29]:
df.head(2)

,bytes_in,bytes_out,creation_time,end_time,src_ip,src_ip_country_code,protocol,response.code,dst_port,dst_ip,rule_names,observation_name,source.meta,source.name,detection_types,connection_time,total_bytes
0,5602,12990,2024-04-25 23:00:00,2024-04-25 23:10:00,147.161.161.82,AE,HTTPS,200,443,10.138.69.97,Suspicious Web Traffic,Adversary Infrastructure Interaction,AWS_VPC_Flow,prod_webserver,waf_rule,600.0,18592
1,30912,18186,2024-04-25 23:00:00,2024-04-25 23:10:00,165.225.33.6,US,HTTPS,200,443,10.138.69.97,Suspicious Web Traffic,Adversary Infrastructure Interaction,AWS_VPC_Flow,prod_webserver,waf_rule,600.0,49098


**Dimensions Tables:**

* ipaddress_dim: {ip_id, src_ip, dst_ip}
* protocol_dim: {protocol_id, protocol}
* country_dim: {country_id, src_ip_country_code}
* port_dim: {port_id, dst_port, protocol}
* names_dim: {name_id, rule_names, observation_name, source.meta, source.name}
* responsecode_dim: {response_id, response.code}
* detectiontype_dim: {detection_id, detection_types}

In [30]:
import os

In [31]:
# Specialized function to create fact & dimension tables

def create_fact_and_dimension_tables(df, dimension_groups, fact_columns, output_dir='/Tables'):
    """
    Function to split a dataframe into fact and dimension tables and add a unique primary key to the fact table.
    
    Parameters:
    df (pd.DataFrame): Original dataframe to split.
    dimension_groups (dict): Dictionary where keys are dimension table names and values are lists of column names.
    fact_columns (list): List of column names that will be part of the fact table.
    output_dir (str): Directory where the CSV files will be saved to.
    
    Returns:
    dict: A dictionary containing the dimension tables and the fact table.
    """
    
    # Create the output directory if it does not exist
    os.makedirs(output_dir, exist_ok=True)    
    
    # Dictionary to store the dimension tables
    dimension_tables = {}
    #fact_table = df --debug
    fact_table = df[fact_columns].copy() #store a copy in fact_table   
   
    for dim_name, dim_columns in dimension_groups.items():
        # Create dimension table & assign unique IDs
        dim_table = df[dim_columns].drop_duplicates().reset_index(drop=True)
        dim_table[f'{dim_name}_id'] = dim_table.index + 1
        
        # Reorder the dimension table to move the unique ID to the first column
        dim_table = dim_table[[f'{dim_name}_id'] + dim_columns]        

        # Store the dimension table in the dictionary
        dimension_tables[dim_name] = dim_table          
        
        # Save the dimension table as CSV file
        dim_table.to_csv(f"{output_dir}/{dim_name}_dim.csv", index=False)
        
        # Debug: Check the columns before the merge
        #print("Fact Table Columns:\n", fact_table.columns.tolist())
        #print("Dimension Table Columns:", dim_table.columns.tolist())        

        # Merge the fact table with the dimension table based on the original dataframe
        fact_table = fact_table.merge(df[dim_columns], how='left', left_index=True, right_index=True)
        
        # Debug purpose
        #print("Fact Table Columns:\n", fact_table.columns.tolist())
                
        fact_table = fact_table.merge(dim_table[[f'{dim_name}_id'] + dim_columns], 
                                       how='left', 
                                       left_on=dim_columns, 
                                       right_on=dim_columns, 
                                       suffixes=('', f'_{dim_name}'))
        
        # Debug purpose
        #print("Fact Table Columns:\n", fact_table.columns.tolist())
        
        # Remove the original dimension columns from the fact table (replace with foreign key)
        fact_table = fact_table.drop(columns=dim_columns)
    
    # Add a surrogate key as the primary key for the fact table
    fact_table['fact_id'] = fact_table.index + 1
    
    # Reorder columns to have the primary key (fact_id) at the front
    columns_order = ['fact_id'] + fact_table.columns.tolist()[:-1]
    fact_table = fact_table[columns_order]
    
    # Save the fact table as a CSV file
    fact_table.to_csv(f"{output_dir}/cw_fact_table.csv", index=False)    
    
    # Return the fact and dimension tables
    return {'fact_table': fact_table, 'dimension_tables': dimension_tables}

**Executing Function to split tables:**

In [32]:
fact_columns = ['bytes_in','bytes_out','creation_time','end_time','connection_time','total_bytes']

dimension_groups = {
    'ipaddress': ['src_ip', 'dst_ip'],
    'country': ['src_ip_country_code'],
    'protocol': ['protocol'],
    'responsecode': ['response.code'],
    'port': ['dst_port'],
    'names': ['rule_names', 'observation_name', 'source.meta', 'source.name'],
    'detectiontype': ['detection_types']
}

tables = create_fact_and_dimension_tables(df, dimension_groups, fact_columns, output_dir='dataset/cloudwatch/Tables')

In [33]:
# Function to print text in bold
def print_bold(text):
    print(f"\033[1m{text}\033[0m")

In [34]:
# Display the results
print_bold("Fact Table Columns:")
print(tables['fact_table'].columns.tolist())

print_bold("\nDimension Tables and Their Columns:")
for dim_name, dim_table in tables['dimension_tables'].items():
    print(f"{dim_name} Dimension Table Columns: {dim_table.columns.tolist()}")

Fact Table Columns:
['fact_id', 'bytes_in', 'bytes_out', 'creation_time', 'end_time', 'connection_time', 'total_bytes', 'ipaddress_id', 'country_id', 'protocol_id', 'responsecode_id', 'port_id', 'names_id', 'detectiontype_id']

Dimension Tables and Their Columns:
ipaddress Dimension Table Columns: ['ipaddress_id', 'src_ip', 'dst_ip']
country Dimension Table Columns: ['country_id', 'src_ip_country_code']
protocol Dimension Table Columns: ['protocol_id', 'protocol']
responsecode Dimension Table Columns: ['responsecode_id', 'response.code']
port Dimension Table Columns: ['port_id', 'dst_port']
names Dimension Table Columns: ['names_id', 'rule_names', 'observation_name', 'source.meta', 'source.name']
detectiontype Dimension Table Columns: ['detectiontype_id', 'detection_types']


***

**References:**
* https://docs.python.org/3/howto/ipaddress.html

**Learnings:**
* How to make use of ipaddress module
* Learn various terminology related to cyberattacks specifically for this dataset
* Using statistics (specfically IQR) + box plots visualisations to check for outliers in transformation
* Writing an advanced function to automatically split dataframe into fact & dimension tables

**Possible improvements:**
* Drop IPs if not valid
* Suspicious flags: Create binary flags (1 or 0) for rule_names that are suspicious/non-suspicious.
* De-bug the double merge, I accidentally made it work and I know what's wrong, just haven't figure out the actual root cause.
* Will debug it again. 

**Why modeling approach? (F&T):**
* Scalability: The fact table contains only foreign keys and numeric facts, reducing data duplication and improving query performance.
* Modularity: The function allows flexibility in selecting which columns to include in dimension tables and which ones to keep in the fact table.
* Reusability: The function can be reused across multiple datasets, making it easier to manage the process of data normalization.

***

### Why Dimensional Modeling (Star Schema) Over 3NF?
```
1. Simplified Queries: 
The star schema allows you to perform simpler queries for reporting and analytics 
(e.g., grouping by protocol, or finding the most common attack sources).

2. Optimized for OLAP: 
Dimensional models are optimized for analytics, easier to create dashboards & run aggregate queries efficiently.

3. Reduces Joins in Reporting: 
Instead of spreading data across multiple normalized tables (e.g. 3NF), the fact table can be used as the central source
for the metrics, with minimal joins required for additional context from dimension tables.

4. Better for Aggregation: 
Dimensional models work well with aggregations, like total traffic by country, 
top IP addresses involved in suspicious activity, and most common detection rules.
```

***

**Possible SQL-Analysis:**
```
* Country Aggregation: Query for summarizing how often connections from each country appear. 
This can help detect patterns in malicious traffic originating from specific regions.

* Connection Frequency: Aggregate the data to calculate how often a specific src_ip or dst_ip appears.
Which might indicate frequent malicious behavior.

* Time-Based Analysis: Extract useful time features from creation_time, such as the hour, day of the week, and month, to analyze patterns (e.g., Are there more suspicious connections at certain times?).
```